In [1]:
import json
import math
import escher
import cobra
import cobrakbase
import modelseedpy_escher
from cobrakbase.core import KBaseFBAModel
from modelseedpy_escher.core import EscherMap

cobrakbase 0.2.8


In [2]:
kbase = cobrakbase.KBaseAPI(dev=True)

In [3]:
escher_seed = modelseedpy_escher.EscherManager(escher)

In [4]:
params = {"column_size": 3,
        "report_height": 800,
        "grid_maps": [{
            "map_id": "custom",
            "user_map_id": ["37534/133/1"],
            "model_id": "37534/89/1",
            "model_compartment_id": "c0",
            "model_alias": "",
            "export_full_model": 0,
            "translate_map": 1,
            "object_ids": [],
            "cpd_abundance": [],
            "cpd_abundance_dataset": "",
            "gene_expression": [],
            "gene_expression_dataset": ""
        }]}

In [5]:
%run ../lib/kb_escher/kb_escher_app.py
kb_escher = KBaseEscher(params, kbase, escher_seed)
em = kb_escher.build()

{'map_id': 'custom', 'user_map_id': ['37534/133/1'], 'model_id': '37534/89/1', 'model_compartment_id': 'c0', 'model_alias': '', 'export_full_model': 0, 'translate_map': 1, 'object_ids': [], 'cpd_abundance': [], 'cpd_abundance_dataset': '', 'gene_expression': [], 'gene_expression_dataset': ''}
grid_cell {'map_id': 'custom.JCVISyn3map', 'model_id': '37534/89/1', 'cmp': 'c0', 'alias': '0', 'adapt': True, 'rxn_data': None, 'cpd_data': None, 'gene_data': None, 'export_full_model': False}
adapt_map_to_model
c0
c0


In [6]:
em.display_in_notebook()

In [6]:
def build_grid_cell2(escher_map, fbamodel, cmp_id = 'c0', alias = None, adapt = True):
    em = adapt_map_to_model2(escher_map, cmp_id, str(alias), fbamodel, adapt)
    return em

def adapt_map_to_model2(em, cmp_id, suffix, fbamodel, adapt=True):
    print('adapt_map_to_model')
    em = em.clone()
    #adapt map to compartment
    print(cmp_id)
    if adapt:
        return cmp_id, em, '0'
        move_to_compartment2(cmp_id, em, '0')
    print(cmp_id)
    map_cpd_set = set()
    map_rxn_set = set()
    for node in em.nodes:
        if node['node_type'] == 'metabolite':
            map_cpd_set.add(node['bigg_id'])
    for rxn_uid in em.escher_graph['reactions']:
        rxn_node = em.escher_graph['reactions'][rxn_uid]
        map_rxn_set.add(rxn_node['bigg_id'])

    model_cpds = set(map(lambda x: x.id, fbamodel.metabolites))
    model_rxns = set(map(lambda x: x.id, fbamodel.reactions))

    map_cpd_delete = map_cpd_set - model_cpds
    map_rxn_delete = map_rxn_set - model_rxns

    #remove metabolites / reactions
    em.delete_reactions(map_rxn_delete)
    em.delete_metabolites(map_cpd_delete)

    #add suffix
    cpd_remap = dict(map(lambda x : (x, x + '@' + suffix), map_cpd_set))
    rxn_remap = dict(map(lambda x : (x, x + '@' + suffix), map_rxn_set))
    em.swap_ids(cpd_remap, rxn_remap)
    #update rev
    #TODO: I WAS HERE!

    return em

for grid_cell in kb_escher.em_data:
    print('grid_cell', grid_cell)
    escher_map = kb_escher.base_maps[grid_cell['map_id']]
    fbamodel = kb_escher.model_cache[grid_cell['model_id']]
    cmp_id = grid_cell['cmp']
    alias = grid_cell['alias']
    adapt = grid_cell['adapt']
    em_cell = build_grid_cell2(escher_map, fbamodel, cmp_id=cmp_id, alias=alias, adapt=adapt)

grid_cell {'map_id': 'custom.JCVISyn3map', 'model_id': '37534/89/1', 'cmp': 'c0', 'alias': '0', 'adapt': True, 'rxn_data': None, 'cpd_data': None, 'gene_data': None, 'export_full_model': False}
adapt_map_to_model
c0


In [64]:
em.clone()

<EscherMap JCVISyn3map at 0x7fec90b15e50>

In [63]:
cmp_id, em, cmp_index = em_cell

In [85]:
em2 = em.clone()
em2.add_uid_to_reaction_metabolites()
1

1

In [110]:
def add_uid_to_reaction_metabolites2(em):
    node_uid_map = metabolite_uid_map(em)

    for rnode in em.reactions:
        node_uid = rnode['uid']
        met_id_to_uid = {}
        for s_uid in rnode['segments']:
            s = rnode['segments'][s_uid]
            from_node_id = rnode['segments'][s_uid]['from_node_id']
            to_node_id = rnode['segments'][s_uid]['to_node_id']
            from_node = em.escher_graph['nodes'][from_node_id]
            to_node = em.escher_graph['nodes'][to_node_id]
            if from_node['node_type'] == 'metabolite':
                if not from_node['bigg_id'] in met_id_to_uid:
                    met_id_to_uid[from_node['bigg_id']] = from_node_id
                else:
                    print('!!!', from_node['bigg_id'])
            if to_node['node_type'] == 'metabolite':
                if not to_node['bigg_id'] in met_id_to_uid:
                    met_id_to_uid[to_node['bigg_id']] = to_node_id
                else:
                    print('!!!', to_node['bigg_id'])

        for m in rnode['metabolites']:
            if m['bigg_id'] in met_id_to_uid:
                m['node_uid'] = met_id_to_uid[m['bigg_id']]

        #print(met_id_to_uid)
        #print(rnode)
        #break

    return node_uid_map

def add_uid_to_reaction_metabolites3(em, rnode, node_uid_map):
    met_id_to_uid = {}
    for s_uid in rnode['segments']:
        s = rnode['segments'][s_uid]
        from_node_id = rnode['segments'][s_uid]['from_node_id']
        to_node_id = rnode['segments'][s_uid]['to_node_id']
        from_node = em.escher_graph['nodes'][from_node_id]
        to_node = em.escher_graph['nodes'][to_node_id]
        if from_node['node_type'] == 'metabolite':
            if not from_node['bigg_id'] in met_id_to_uid:
                met_id_to_uid[from_node['bigg_id']] = from_node_id
            else:
                print('!!!', from_node['bigg_id'])
        if to_node['node_type'] == 'metabolite':
            if not to_node['bigg_id'] in met_id_to_uid:
                met_id_to_uid[to_node['bigg_id']] = to_node_id
            else:
                print('!!!', to_node['bigg_id'])
    for m in rnode['metabolites']:
        if m['bigg_id'] in met_id_to_uid:
            m['node_uid'] = met_id_to_uid[m['bigg_id']]

em2 = em.clone()
node_uid_map = metabolite_uid_map(em2)
#add_uid_to_reaction_metabolites2(em2) == em2.add_uid_to_reaction_metabolites()
o = list(filter(lambda x: x['bigg_id'] == 'rxn35242_c0', em2.reactions))[0]
from pprint import pprint
pprint(o['metabolites'])
add_uid_to_reaction_metabolites3(em2, o, node_uid_map)
pprint(o['metabolites'])

[{'bigg_id': 'cpd00061_c0', 'coefficient': -1},
 {'bigg_id': 'cpd00067_c0', 'coefficient': -1},
 {'bigg_id': 'cpd00020_c0', 'coefficient': 1},
 {'bigg_id': 'cpd00008_c0', 'coefficient': -1},
 {'bigg_id': 'cpd00002_c0', 'coefficient': 1}]
[{'bigg_id': 'cpd00061_c0', 'coefficient': -1, 'node_uid': '2075338'},
 {'bigg_id': 'cpd00067_c0', 'coefficient': -1},
 {'bigg_id': 'cpd00020_c0', 'coefficient': 1, 'node_uid': '2076584'},
 {'bigg_id': 'cpd00008_c0', 'coefficient': -1, 'node_uid': '2075491'},
 {'bigg_id': 'cpd00002_c0', 'coefficient': 1, 'node_uid': '2075493'}]


In [115]:
def detect_current_compartment(n):
    id = n['bigg_id']
    if 'compartment' in n:
        return id, n['compartment']
    if '_' in id:
        p = id.split('_')
        base = '_'.join(p[:-1])
        return base, p[-1]
    return None, None
def detect_compartment_index(cmp):
    if cmp == None or len(cmp) == 1:
        return None
    return cmp[1:]
def add_compartment(em, node_uid_cmp):
    for node_uid in node_uid_cmp:
        if node_uid in em.escher_graph['nodes']:
            node = em.escher_graph['nodes'][node_uid]
            node['compartment'] = node_uid_cmp[node_uid]
def move_to_compartment3(cmp_id, em, cmp_index):
    em.add_uid_to_reaction_metabolites()
    node_uid_cmp = {}
    node_uid_id = {}
    for n in em.metabolites:
        node_uid = n['uid']
        target_cmp = cmp_id
        base, node_cmp = detect_current_compartment(n)
        if not node_cmp == None:
            node_cmp_index = detect_compartment_index(node_cmp)
            if node_cmp_index == None:
                node_cmp += str(cmp_index)
            target_cmp = node_cmp
        #print(n['bigg_id'], target_cmp)
        if not 'compartment' in n:
            node_uid_cmp[node_uid] = target_cmp
            if not base == None:
                n['bigg_id'] = base + '_' + target_cmp
            else:
                n['bigg_id'] += '_' + target_cmp
            node_uid_id[node_uid] = n['bigg_id']
    add_compartment(em, node_uid_cmp)
    for rnode in em.reactions:
        rxn_uid = rnode['uid']
        target_cmp = cmp_id
        base, node_cmp = detect_current_compartment(rnode)
        if not node_cmp == None:
            node_cmp_index = detect_compartment_index(node_cmp)
            if node_cmp_index == None:
                node_cmp += str(cmp_index)
            target_cmp = node_cmp
        for o in rnode['metabolites']:
            if 'node_uid' in o:
                o['bigg_id'] = node_uid_id[o['node_uid']]
         
        if not base == None:
            rnode['bigg_id'] = base + '_' + target_cmp
        else:
            rnode['bigg_id'] += '_' + target_cmp
    return node_uid_cmp
move_to_compartment3(cmp_id, em.clone(), cmp_index)

{'2075338': 'c0',
 '2075354': 'c0',
 '2075355': 'c0',
 '2075382': 'c0',
 '2075395': 'c0',
 '2075396': 'c0',
 '2075406': 'c0',
 '2075428': 'c0',
 '2075460': 'c0',
 '2075461': 'c0',
 '2075462': 'c0',
 '2075464': 'c0',
 '2075465': 'c0',
 '2075491': 'c0',
 '2075493': 'c0',
 '2075576': 'c0',
 '2075602': 'c0',
 '2075603': 'c0',
 '2075638': 'c0',
 '2075696': 'c0',
 '2076584': 'c0',
 '2076589': 'c0',
 '2076590': 'c0',
 '2076591': 'c0',
 '2076592': 'c0',
 '2076596': 'c0',
 '2076597': 'c0',
 '2076598': 'c0',
 '2076599': 'c0',
 '2076615': 'c0',
 '2077723': 'e0',
 '2077727': 'c0',
 '2077728': 'c0',
 '2077729': 'c0',
 '2077730': 'c0',
 '2077731': 'c0',
 '2077732': 'c0',
 '2077788': 'c0',
 '2077789': 'c0',
 '2077889': 'c0',
 '2077893': 'c0',
 '2077894': 'c0',
 '2077895': 'c0',
 '2077896': 'c0',
 '2077897': 'c0',
 '2077898': 'e0',
 '2077902': 'c0',
 '2077903': 'c0',
 '2077904': 'c0',
 '2077905': 'c0',
 '2077909': 'c0',
 '2077910': 'c0',
 '2077911': 'c0',
 '2077912': 'c0',
 '2077913': 'c0',
 '2077917'

In [22]:
#kb_escher.get_chemical_abundance_data(kb_escher.object_cache['55494/31/6'], 'Glucose - Exponential')

In [24]:
row_attributemapping = kb_escher.get_object_from_ref(o['row_attributemapping_ref'])

In [27]:
row_attributemapping['attributes']

[{'attribute': 'name', 'source': 'upload'},
 {'attribute': 'modelseed',
  'attribute_ont_id': 'Custom:Term',
  'attribute_ont_ref': 'KbaseOntologies/Custom',
  'source': 'ModelSEED'}]

In [41]:
params = {
        "column_size": 3,
        "report_height": 800,
        "grid_maps": [{
            "map_id": "custom",
            "user_map_id": ["55494/68/2"],
            "model_id": "56171/42/2", #56171/42/1 no maxfor #56171/41/15
            "model_compartment_id": "c0",
            "model_alias": "modelAlias",
            "object_ids": ["55494/40/12"],
            "cpd_abundance": ["55494/31/6"],
            "cpd_abundance_dataset": "Glucose - Exponential",
            "gene_expression": ["55494/27/1"],
            "gene_expression_dataset": ""
        }]
    }

In [42]:
%run ../lib/kb_escher/kb_escher_app.py
kb_escher = KBaseEscher(params, kbase, escher_seed)
em = kb_escher.build()

{'map_id': 'custom', 'user_map_id': ['55494/68/2'], 'model_id': '56171/42/2', 'model_compartment_id': 'c0', 'model_alias': 'modelAlias', 'object_ids': ['55494/40/12'], 'cpd_abundance': ['55494/31/6'], 'cpd_abundance_dataset': 'Glucose - Exponential', 'gene_expression': ['55494/27/1'], 'gene_expression_dataset': ''}
grid_cell {'map_id': 'custom.AromaticAminoAcidBiosynthesisMap', 'model_id': '56171/42/2', 'cmp': 'c0', 'alias': 'modelAlias', 'adapt': False, 'rxn_data': '55494/40/12', 'cpd_data': ('55494/31/6', 'Glucose - Exponential'), 'gene_data': None, 'export_full_model': False}
adapt_map_to_model
c0
c0


In [35]:
kb_escher

In [28]:
em.display_in_notebook()

In [20]:
next(em.nodes)

{'bigg_id': 'cpd00216',
 'label_x': -858.2646744409196,
 'label_y': 1195.1824006238257,
 'name': 'Chorismate',
 'node_is_primary': 1,
 'node_type': 'metabolite',
 'x': -706.8099958896637,
 'y': 1185.572006225586,
 'uid': '146'}

In [21]:
%run ../lib/kb_escher/kb_escher_app.py
kb_escher = KBaseEscher(params, api, escher_seed)
kb_escher.build()
#kb_escher.export('/Users/fliu/workspace/jupyter/python3/kbase/kb_escher/data/html/data/')

{'map_id': 'custom', 'user_map_id': ['55494/68/2'], 'model_id': '55494/17/15', 'model_compartment_id': 'c0', 'model_alias': 'modelAlias', 'object_ids': ['55494/40/12'], 'cpd_abundance': ['55494/31/6'], 'cpd_abundance_dataset': 'Glucose - Exponential', 'gene_expression': ['55494/27/1'], 'gene_expression_dataset': ''}
grid_cell {'map_id': 'custom.AromaticAminoAcidBiosynthesisMap', 'model_id': '55494/17/15', 'cmp': 'c0', 'alias': 'modelAlias', 'adapt': False, 'rxn_data': '55494/40/12', 'cpd_data': ('55494/31/6', 'Glucose - Exponential'), 'gene_data': None, 'export_full_model': False}
adapt_map_to_model
c0
c0


<EscherMap custom map at 0x7ff6ca2ac210>

In [10]:
%run ../lib/kb_escher/kb_escher_app.py
params = {
        "column_size": 1,
        "gene_mode": 0,
        "report_height": 800,
        "grid_maps": [{
            "map_id": "custom",
            "user_map_id": ["82295/23/1"],
            "model_id": "82295/20/1",
            "model_compartment_id": "c0",
            "model_alias": "ANME",
            "export_full_model": 0,
            "translate_map": 1,
            "object_ids": [],
            "cpd_abundance": [],
            "cpd_abundance_dataset": "",
            "gene_expression": [],
            "gene_expression_dataset": ""
        }]
    }
kb_escher = KBaseEscher(params, api, escher_seed)
em = kb_escher.build()

{'map_id': 'custom', 'user_map_id': ['82295/23/1'], 'model_id': '82295/20/1', 'model_compartment_id': 'c0', 'model_alias': 'ANME', 'export_full_model': 0, 'translate_map': 1, 'object_ids': [], 'cpd_abundance': [], 'cpd_abundance_dataset': '', 'gene_expression': [], 'gene_expression_dataset': ''}


ServerError: JSONRPCError: -32500. Object 23 (name JCVISyn3map-deleted-1612383453165) in workspace 82295 (name filipeliu:narrative_1612368108584) has been deleted
us.kbase.workspace.database.exceptions.DeletedObjectException: Object 23 (name JCVISyn3map-deleted-1612383453165) in workspace 82295 (name filipeliu:narrative_1612368108584) has been deleted
	at us.kbase.workspace.database.mongo.MongoWorkspaceDB.queryObjects(MongoWorkspaceDB.java:2926)
	at us.kbase.workspace.database.mongo.MongoWorkspaceDB.resolveObjectIDs(MongoWorkspaceDB.java:2858)
	at us.kbase.workspace.database.mongo.MongoWorkspaceDB.getObjectInformation(MongoWorkspaceDB.java:2811)
	at us.kbase.workspace.database.Workspace.getObjectInformation(Workspace.java:1388)
	at us.kbase.workspace.kbase.WorkspaceServerMethods.getObjectInformation(WorkspaceServerMethods.java:406)
	at us.kbase.workspace.WorkspaceServer.getObjectInfo3(WorkspaceServer.java:1018)
	at sun.reflect.GeneratedMethodAccessor87.invoke(Unknown Source)
	at sun.re

In [ ]:
em.display_in_notebook()

In [34]:
kb_escher.global_cpd_data

{'cpd00041_c0@modelAlias': 2926,
 'cpd00023_c0@modelAlias': 1459,
 'cpd00066_c0@modelAlias': 199,
 'cpd00143_c0@modelAlias': 546,
 'cpd00009_c0@modelAlias': 146564,
 'cpd00012_c0@modelAlias': 15166,
 'cpd00054_c0@modelAlias': 14415,
 'cpd00065_c0@modelAlias': 333,
 'cpd00069_c0@modelAlias': 3058}

In [16]:
params = {
        "column_size": 3,
        "report_height": 800,
        "grid_maps": [{
            "map_id": "custom",
            "user_map_id": ["55494/68/2"],
            "model_id": "55494/17/15",
            "model_compartment_id": "c0",
            "model_alias": "modelAlias",
            "object_ids": ["55494/40/12"],
            "cpd_abundance": ["55494/31/6"],
            "cpd_abundance_dataset": "Glucose - Exponential",
            "gene_expression": ["55494/27/1"],
            "gene_expression_dataset": ""
        }]
    }
params =  {
        "column_size": 1,
        "gene_mode": 0,
        "grid_maps": [{
            "cpd_abundance": [],
            "cpd_abundance_dataset": "",
            "export_full_model": 1,
            "gene_expression": [],
            "gene_expression_dataset": "",
            "map_id": "custom",
            "model_alias": "",
            "model_compartment_id": "c0",
            "model_id": "60965/7/1",
            "object_ids": [],
            "user_map_id": ["60965/8/1"]
        }],
        "report_height": 800
    }

In [17]:
kb_escher = KBaseEscher(params, api, escher_seed)
kb_escher.build()

{'cpd_abundance': [], 'cpd_abundance_dataset': '', 'export_full_model': 1, 'gene_expression': [], 'gene_expression_dataset': '', 'map_id': 'custom', 'model_alias': '', 'model_compartment_id': 'c0', 'model_id': '60965/7/1', 'object_ids': [], 'user_map_id': ['60965/8/1']}
{'map_id': 'custom.Central_Metabolism', 'model_id': '60965/7/1', 'cmp': 'c0', 'alias': '0', 'rxn_data': None, 'cpd_data': None, 'gene_data': None, 'export_full_model': True}
!!! cpd00011_c0


ValueError: not enough values to unpack (expected 2, got 1)